In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import random
import glob
from sklearn.model_selection import train_test_split
import tensorflow_addons as tfa
from tensorflow.keras import applications
from tensorflow.keras import callbacks
from sklearn.neighbors import KNeighborsClassifier
from tensorflow.keras import callbacks

CALLBACK = callbacks.EarlyStopping(monitor="val_loss", mode="min", verbose=1, patience=15, restore_best_weights=True)

c:\users\kajoj\appdata\local\programs\python\python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.8.0 and strictly below 2.11.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.11.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [10]:
import pickle

pickle_in = open("X_train_128.pickle", "rb")
X_train = pickle.load(pickle_in)

pickle_in = open("Y_train_128.pickle", "rb")
Y_train = pickle.load(pickle_in)

pickle_in = open("X_test_128.pickle", "rb")
X_test = pickle.load(pickle_in)

pickle_in = open("Y_test_128.pickle", "rb")
Y_test = pickle.load(pickle_in)

In [6]:
model = tf.keras.Sequential([
    tf.keras.applications.ResNet152V2(weights=None, input_shape=X_train.shape[1:]),
    tf.keras.layers.Flatten(name="flatten"),
    tf.keras.layers.Dense(256, activation="relu", name="dense1"),
    tf.keras.layers.BatchNormalization(name="norm1"),
    tf.keras.layers.Dense(256, activation=None, name="dense4"),
    tf.keras.layers.Lambda(lambda x: tf.math.l2_normalize(x, axis=1), name="output1")
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tfa.losses.TripletSemiHardLoss())

In [7]:
history = model.fit(
    X_train, Y_train, batch_size=32,
    epochs=200, validation_split = 0.3, callbacks=CALLBACK)

Epoch 1/200
187/187 [==============================] - 556s 3s/step - loss: 0.8578 - val_loss: 0.7491
Epoch 2/200
187/187 [==============================] - 524s 3s/step - loss: 0.8495 - val_loss: 0.8354
Epoch 3/200
187/187 [==============================] - 523s 3s/step - loss: 0.7629 - val_loss: 0.9170
Epoch 4/200
187/187 [==============================] - 523s 3s/step - loss: 0.6961 - val_loss: 0.9517
Epoch 5/200
187/187 [==============================] - 523s 3s/step - loss: 0.6493 - val_loss: 0.7516
Epoch 6/200
187/187 [==============================] - 522s 3s/step - loss: 0.6411 - val_loss: 0.8841
Epoch 7/200
187/187 [==============================] - 522s 3s/step - loss: 0.8309 - val_loss: 0.7393
Epoch 8/200
187/187 [==============================] - 524s 3s/step - loss: 0.7353 - val_loss: 0.9832
Epoch 9/200
187/187 [==============================] - 526s 3s/step - loss: 0.6685 - val_loss: 1.0380
Epoch 10/200
187/187 [==============================] - 527s 3s/step - loss: 0.674

In [11]:
train_embs = model.predict(X_train)
test_embs = model.predict(X_test)

115/115 [==============================] - 62s 540ms/step


In [12]:
import io
import tensorflow_datasets as tfds

np.savetxt("vecs-ResNet152V2-d256-d128.tsv", test_embs, delimiter='\t')

out_m = io.open('meta-ResNet152V2-d256-d128.tsv', 'w', encoding='utf-8')
[out_m.write(str(x) + "\n") for x in Y_test]
out_m.close()


try:
  from google.colab import files
  files.download('vecs-ResNet152V2-d256-d256.tsv')
  files.download('meta-ResNet152V2-d256-d256.tsv')
except:
  pass

In [13]:
knn = KNeighborsClassifier(n_neighbors=15)
knn.fit(train_embs, Y_train)
y_test_pred_knn = knn.predict_proba(test_embs)


In [14]:
predict = []
for pred in y_test_pred_knn:
    if(pred[0]>=pred[1]):
        predict.append(0)
    else:
        predict.append(1)

In [15]:
TP = 0
FP = 0
TN = 0
FN = 0

for i in range (len(predict)):
    if predict[i] == 0 and Y_test[i] == 0:
        TN += 1
    elif predict[i] == 1 and Y_test[i] == 1:
        TP += 1
    elif predict[i] == 0 and Y_test[i] == 1:
        FN += 1
    elif predict[i] == 1 and Y_test[i] == 0:
        FP += 1

In [16]:
czulosc = TP / (TP + FN)
swoistosc = TN / (TN + FP)

print(czulosc)
print(swoistosc)

0.8333333333333334
0.9391929242675512


In [17]:
model.save('ResNet152V2-d256-d256')

INFO:tensorflow:Assets written to: ResNet152V2-d256-d256\assets


INFO:tensorflow:Assets written to: ResNet152V2-d256-d256\assets
